In [1]:
import numpy as np
import pandas as pd
from hmmlearn import hmm

Data:

In [2]:
angles_control_odor = pd.read_csv('C:/Users/viky/Desktop/TFM/decision making/Navigation - Decision Making/thG@Uempty/n_1ul1000EA_600s@n/angles_control_odor.csv')
angles_control_free = pd.read_csv('C:/Users/viky/Desktop/TFM/decision making/Navigation - Decision Making/thG@Uempty/n_freeNavigation_600s@n/angles_control_free.csv')
angles_aSynA53T_odor = pd.read_csv('C:/Users/viky/Desktop/TFM/decision making/Navigation - Decision Making/thG@UaSynA53T/n_1ul1000EA_600s@n/angles_asyn_odor.csv')
angles_aSynA53T_free = pd.read_csv('C:/Users/viky/Desktop/TFM/decision making/Navigation - Decision Making/thG@UaSynA53T//n_freeNavigation_600s@n/angles_asyn_free.csv')
angles_G2019S_odor = pd.read_csv('C:/Users/viky/Desktop/TFM/decision making/Navigation - Decision Making/thG@UG2019S/n_1ul1000EA_600s@n/angles_LRRK2_odor.csv')
angles_G2019S_free = pd.read_csv('C:/Users/viky/Desktop/TFM/decision making/Navigation - Decision Making/thG@UG2019S/n_freeNavigation_600s@n/angles_LRRK2_free.csv')

Round angles:

In [3]:
angles_control_odor['angle (degrees)'] = angles_control_odor['angle (degrees)'].round().astype(int)
angles_control_free['angle (degrees)'] = angles_control_free['angle (degrees)'].round().astype(int)
angles_aSynA53T_odor['angle (degrees)'] = angles_aSynA53T_odor['angle (degrees)'].round().astype(int)
angles_aSynA53T_free['angle (degrees)'] = angles_aSynA53T_free['angle (degrees)'].round().astype(int)
angles_G2019S_odor['angle (degrees)'] = angles_G2019S_odor['angle (degrees)'].round().astype(int)
angles_G2019S_free['angle (degrees)'] = angles_G2019S_free['angle (degrees)'].round().astype(int)

# HMM first population: emptyUAS_odor

## Transition matrix:

225º-315º, 45º-135º, 135º-225º y 315º-45º (heading odor, opposite to odor, heading top, heading bottom). 

In [4]:
# Definir los rangos de ángulos
heading_odor = (225, 315)
opposite_odor = (45, 135)
heading_top = (135, 225)
heading_bottom = (315, 45)

# Inicializar contadores para contar las transiciones entre rangos y mantenerse en el mismo rango
transiciones_heading_odor_a_opposite_odor = 0
transiciones_heading_odor_a_heading_top = 0
transiciones_heading_odor_a_heading_bottom = 0
transiciones_heading_odor_a_heading_odor = 0 
transiciones_opposite_odor_a_heading_odor = 0
transiciones_opposite_odor_a_heading_top = 0
transiciones_opposite_odor_a_heading_bottom = 0
transiciones_opposite_odor_a_opposite_odor = 0 
transiciones_heading_top_a_heading_odor = 0
transiciones_heading_top_a_opposite_odor = 0
transiciones_heading_top_a_heading_bottom = 0
transiciones_heading_top_a_heading_top = 0 
transiciones_heading_bottom_a_heading_odor = 0
transiciones_heading_bottom_a_opposite_odor = 0
transiciones_heading_bottom_a_heading_top = 0
transiciones_heading_bottom_a_heading_bottom = 0  

# Iterar a través de la lista de ángulos
for i in range(len(angles_control_odor) - 1):
    angulo_actual = angles_control_odor['angle (degrees)'][i]
    angulo_siguiente = angles_control_odor['angle (degrees)'][i + 1]
    
    # Verificar las transiciones y contarlas
    if heading_odor[0] <= angulo_actual < heading_odor[1]:
        if opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_odor_a_opposite_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_odor_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_odor_a_heading_bottom += 1
        elif heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_odor_a_heading_odor += 1
    elif opposite_odor[0] <= angulo_actual < opposite_odor[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_opposite_odor_a_heading_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_opposite_odor_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_opposite_odor_a_heading_bottom += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_opposite_odor_a_opposite_odor += 1
    elif heading_top[0] <= angulo_actual < heading_top[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_top_a_heading_odor += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_top_a_opposite_odor += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_top_a_heading_bottom += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_top_a_heading_top += 1
    elif heading_bottom[0] <= angulo_actual <= 360 or 0 <= angulo_actual < heading_bottom[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_bottom_a_heading_odor += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_bottom_a_opposite_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_bottom_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_bottom_a_heading_bottom += 1

# Calcular las probabilidades de transición
heading_odor_transitions = (transiciones_heading_odor_a_heading_bottom, transiciones_heading_odor_a_heading_odor,
                                              transiciones_heading_odor_a_heading_top,transiciones_heading_odor_a_opposite_odor)
total_transiciones_heading_odor = sum(heading_odor_transitions)

opposite_odor_transitions = (transiciones_opposite_odor_a_heading_bottom, transiciones_opposite_odor_a_heading_odor,
                                              transiciones_opposite_odor_a_heading_top,transiciones_opposite_odor_a_opposite_odor)
total_transiciones_opposite_odor = sum(opposite_odor_transitions)

heading_top_transitions = (transiciones_heading_top_a_heading_bottom, transiciones_heading_top_a_heading_odor,
                                              transiciones_heading_top_a_heading_top,transiciones_heading_top_a_opposite_odor)
total_transiciones_heading_top = sum(heading_top_transitions)

heading_bottom_transitions = (transiciones_heading_bottom_a_heading_bottom, transiciones_heading_bottom_a_heading_odor,
                                              transiciones_heading_bottom_a_heading_top,transiciones_heading_bottom_a_opposite_odor)
total_transiciones_heading_bottom = sum(heading_bottom_transitions)

# Calcular las probabilidades de transición dividiendo el número de transiciones entre el total de transiciones

probabilidad_heading_odor_a_opposite_odor = transiciones_heading_odor_a_opposite_odor / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_top = transiciones_heading_odor_a_heading_top / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_bottom = transiciones_heading_odor_a_heading_bottom / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_odor = transiciones_heading_odor_a_heading_odor / total_transiciones_heading_odor

probabilidad_opposite_odor_a_heading_odor = transiciones_opposite_odor_a_heading_odor / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_heading_top = transiciones_opposite_odor_a_heading_top / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_heading_bottom = transiciones_opposite_odor_a_heading_bottom / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_opposite_odor = transiciones_opposite_odor_a_opposite_odor / total_transiciones_opposite_odor

probabilidad_heading_top_a_heading_odor = transiciones_heading_top_a_heading_odor / total_transiciones_heading_top
probabilidad_heading_top_a_opposite_odor = transiciones_heading_top_a_opposite_odor / total_transiciones_heading_top
probabilidad_heading_top_a_heading_bottom = transiciones_heading_top_a_heading_bottom / total_transiciones_heading_top
probabilidad_heading_top_a_heading_top = transiciones_heading_top_a_heading_top / total_transiciones_heading_top

probabilidad_heading_bottom_a_heading_odor = transiciones_heading_bottom_a_heading_odor/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_opposite_odor = transiciones_heading_bottom_a_opposite_odor/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_heading_top = transiciones_heading_bottom_a_heading_top/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_heading_bottom = transiciones_heading_bottom_a_heading_bottom/total_transiciones_heading_bottom


# Imprimir las probabilidades de transición
print(f"probabilidad_heading_odor_a_opposite_odor: {probabilidad_heading_odor_a_opposite_odor}")
print(f"probabilidad_heading_odor_a_heading_top: {probabilidad_heading_odor_a_heading_top}")
print(f"probabilidad_heading_odor_a_heading_bottom: {probabilidad_heading_odor_a_heading_bottom}")
print(f"probabilidad_heading_odor_a_heading_odor: {probabilidad_heading_odor_a_heading_odor}")

print(f"probabilidad_opposite_odor_a_heading_odor: {probabilidad_opposite_odor_a_heading_odor}")
print(f"probabilidad_opposite_odor_a_heading_top: {probabilidad_opposite_odor_a_heading_top}")
print(f"probabilidad_opposite_odor_a_heading_bottom: {probabilidad_opposite_odor_a_heading_bottom}")
print(f"probabilidad_opposite_odor_a_opposite_odor: {probabilidad_opposite_odor_a_opposite_odor}")

print(f"probabilidad_heading_top_a_heading_odor: {probabilidad_heading_top_a_heading_odor}")
print(f"probabilidad_heading_top_a_opposite_odor: {probabilidad_heading_top_a_opposite_odor}")
print(f"probabilidad_heading_top_a_heading_bottom: {probabilidad_heading_top_a_heading_bottom}")
print(f"probabilidad_heading_top_a_heading_top: {probabilidad_heading_top_a_heading_top}")

print(f"probabilidad_heading_bottom_a_heading_odor: {probabilidad_heading_bottom_a_heading_odor}")
print(f"probabilidad_heading_bottom_a_opposite_odor: {probabilidad_heading_bottom_a_opposite_odor}")
print(f"probabilidad_heading_bottom_a_heading_top: {probabilidad_heading_bottom_a_heading_top}")
print(f"probabilidad_heading_bottom_a_heading_bottom: {probabilidad_heading_bottom_a_heading_bottom}")


probabilidad_heading_odor_a_opposite_odor: 0.0040538536328131435
probabilidad_heading_odor_a_heading_top: 0.04144488571758153
probabilidad_heading_odor_a_heading_bottom: 0.04241715142626436
probabilidad_heading_odor_a_heading_odor: 0.9120841092233409
probabilidad_opposite_odor_a_heading_odor: 0.004981200756759346
probabilidad_opposite_odor_a_heading_top: 0.047872213042124676
probabilidad_opposite_odor_a_heading_bottom: 0.0457168857916038
probabilidad_opposite_odor_a_opposite_odor: 0.9014297004095122
probabilidad_heading_top_a_heading_odor: 0.056927060854553305
probabilidad_heading_top_a_opposite_odor: 0.04044022442813983
probabilidad_heading_top_a_heading_bottom: 0.004596460940871817
probabilidad_heading_top_a_heading_top: 0.898036253776435
probabilidad_heading_bottom_a_heading_odor: 0.05410987195373813
probabilidad_heading_bottom_a_opposite_odor: 0.04337050805452292
probabilidad_heading_bottom_a_heading_top: 0.0046087958433879
probabilidad_heading_bottom_a_heading_bottom: 0.8979108241

## Initial probabilities:

In [7]:
# Inicializa contadores para contar la población inicial en cada rango
initial_heading_odor = 0
initial_opposite_odor = 0
initial_heading_top = 0
initial_heading_bottom = 0

# Itera a través de los eventos en el tiempo 0
for angle in angles_control_odor['angle (degrees)']:
    if heading_odor[0] <= angle < heading_odor[1]:
        initial_heading_odor += 1
    elif opposite_odor[0] <= angle < opposite_odor[1]:
        initial_opposite_odor += 1
    elif heading_top[0] <= angle < heading_top[1]:
        initial_heading_top += 1
    elif heading_bottom[0] <= angle <= 360 or 0 <= angle < heading_bottom[1]:
        initial_heading_bottom += 1

total_events = len(angles_control_odor)

# Calcula la prob de inicio en cada rango
prob_initial_heading_odor = initial_heading_odor / total_events
prob_initial_opposite_odor = initial_opposite_odor / total_events
prob_initial_heading_top = initial_heading_top / total_events
prob_initial_heading_bottom = initial_heading_bottom / total_events

# Imprime las probes de inicio en cada rango
print("prob inicial en heading_odor:", prob_initial_heading_odor)
print("prob inicial en opposite_odor:", prob_initial_opposite_odor)
print("prob inicial en heading_top:", prob_initial_heading_top)
print("prob inicial en heading_bottom:", prob_initial_heading_bottom)


prob inicial en heading_odor: 0.3115463599958928
prob inicial en opposite_odor: 0.2143803265222302
prob inicial en heading_top: 0.23791457028442345
prob inicial en heading_bottom: 0.23615874319745353


## Emission matrix

In [8]:
# Define una función para asignar el estado según las reglas dadas
def asign_orientation(angle):
    if 225 <= angle < 315:
        return 'heading odor'
    elif 45 <= angle < 135:
        return 'opposite to odor'
    elif 135 <= angle < 225:
        return 'heading top'
    else:
        return 'heading bottom'

# Aplica la función para crear una nueva columna 'Estado'
angles_control_odor['Orientation'] = angles_control_odor['angle (degrees)'].apply(asign_orientation)

# Crea un diccionario para almacenar listas de ángulos por estado
angles_per_orientation = {
    'heading odor': [],
    'opposite to odor': [],
    'heading top': [],
    'heading bottom': []
}

# Llena el diccionario con los ángulos correspondientes
for orientation, sub_angles_control_odor in angles_control_odor.groupby('Orientation'):
    angles_per_orientation[orientation] = sub_angles_control_odor['angle (degrees)'].tolist()

all_frecuencies = []

# Calcula las frecuencias absolutas y normaliza
for orientation, angles in angles_per_orientation.items():
    abs_frecuencies = pd.Series(angles).value_counts().reset_index()
    abs_frecuencies.columns = ['Angle', 'Frecuency']
    abs_frecuencies['Normalized frecuency'] = abs_frecuencies['Frecuency'] / len(angles)
    
    all_frecuencies.append({'Orientation': orientation, 'Frequencies': abs_frecuencies})
    
    print(f"Orientation: {orientation}")
    print(abs_frecuencies)
    print("\n")

Orientation: heading odor
    Angle  Frecuency  Normalized frecuency
0     266        807              0.013299
1     259        805              0.013266
2     267        801              0.013200
3     261        799              0.013167
4     263        796              0.013117
..    ...        ...                   ...
85    296        560              0.009228
86    311        556              0.009162
87    307        555              0.009146
88    312        550              0.009063
89    313        521              0.008586

[90 rows x 3 columns]


Orientation: opposite to odor
    Angle  Frecuency  Normalized frecuency
0      48        559              0.013387
1      86        559              0.013387
2      50        536              0.012836
3      81        532              0.012740
4      79        532              0.012740
..    ...        ...                   ...
85    131        394              0.009436
86    113        393              0.009412
87    133       

In [18]:
heading_odor_frec = all_frecuencies[0]
heading_odor_frec = heading_odor_frec['Frequencies']
heading_odor_frec = heading_odor_frec.sort_values(by='Angle')
heading_odor_frec = heading_odor_frec['Normalized frecuency']

heading_bottom_frec = all_frecuencies[3]
heading_bottom_frec = heading_bottom_frec['Frequencies']
heading_bottom_frec = heading_bottom_frec.sort_values(by='Angle')
heading_bottom_frec = heading_bottom_frec['Normalized frecuency']

opposite_to_odor_frec = all_frecuencies[1]
opposite_to_odor_frec = opposite_to_odor_frec['Frequencies']
opposite_to_odor_frec = opposite_to_odor_frec.sort_values(by='Angle')
opposite_to_odor_frec = opposite_to_odor_frec['Normalized frecuency']

heading_top_frec = all_frecuencies[2]
heading_top_frec = heading_top_frec['Frequencies']
heading_top_frec = heading_top_frec.sort_values(by='Angle')
heading_top_frec = heading_top_frec['Normalized frecuency']

ceros_adicionales90 = np.zeros(90)
ceros_adicionales180 = np.zeros(180)
ceros_adicionales271 = np.zeros(271)
ceros_adicionales181 = np.zeros(181)

heading_odor_frec = np.concatenate((heading_odor_frec, ceros_adicionales271))
heading_bottom_frec = np.concatenate((ceros_adicionales90,heading_bottom_frec,ceros_adicionales180))
opposite_to_odor_frec = np.concatenate((ceros_adicionales181,opposite_to_odor_frec,ceros_adicionales90))
heading_top_frec = np.concatenate((ceros_adicionales271,heading_top_frec))

## HMM

In [90]:
num_estados = 4  # components

startprob = np.array([0.3115463599958928, 0.23615874319745353, 0.2143803265222302, 0.23791457028442345])
prob_transiciones = np.array([
    [0.9120841092233409, 0.04241715142626436, 0.0040538536328131435, 0.04144488571758153],
    [0.05410987195373813, 0.8979108241483511, 0.04337050805452292, 0.0046087958433879],
    [0.004981200756759346, 0.0457168857916038, 0.9014297004095122, 0.047872213042124676],
    [0.056927060854553305, 0.004596460940871817, 0.04044022442813983, 0.898036253776435]
])
prob_emisiones = np.array([heading_odor_frec,heading_bottom_frec,opposite_to_odor_frec,heading_top_frec])

model = hmm.MultinomialHMM(n_components=num_estados,n_trials=1)
model.startprob_ = startprob
model.transmat_ = prob_transiciones
model.emissionprob_ = prob_emisiones


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


In [61]:
def generar_secuencia_estable(modelo_hmm, num_pasos=1000000):
    secuencia = []
    estado_actual = np.random.choice(modelo_hmm.n_components, p=modelo_hmm.startprob_)  # Estado inicial aleatorio
    for _ in range(num_pasos):
        # Generar una nueva observación en función del estado actual
        observacion = np.random.choice(modelo_hmm.emissionprob_.shape[1], p=modelo_hmm.emissionprob_[estado_actual])
        secuencia.append(observacion)
        # Realizar una transición de estado basada en las probabilidades de transición
        estado_actual = np.random.choice(modelo_hmm.n_components, p=modelo_hmm.transmat_[estado_actual])
    return secuencia

secuencia_estable = generar_secuencia_estable(model)

# Probabilities  emptyUAS_free

In [10]:
# Definir los rangos de ángulos
heading_odor = (225, 315)
opposite_odor = (45, 135)
heading_top = (135, 225)
heading_bottom = (315, 45)

# Inicializar contadores para contar las transiciones entre rangos y mantenerse en el mismo rango
transiciones_heading_odor_a_opposite_odor = 0
transiciones_heading_odor_a_heading_top = 0
transiciones_heading_odor_a_heading_bottom = 0
transiciones_heading_odor_a_heading_odor = 0 
transiciones_opposite_odor_a_heading_odor = 0
transiciones_opposite_odor_a_heading_top = 0
transiciones_opposite_odor_a_heading_bottom = 0
transiciones_opposite_odor_a_opposite_odor = 0 
transiciones_heading_top_a_heading_odor = 0
transiciones_heading_top_a_opposite_odor = 0
transiciones_heading_top_a_heading_bottom = 0
transiciones_heading_top_a_heading_top = 0 
transiciones_heading_bottom_a_heading_odor = 0
transiciones_heading_bottom_a_opposite_odor = 0
transiciones_heading_bottom_a_heading_top = 0
transiciones_heading_bottom_a_heading_bottom = 0  

# Iterar a través de la lista de ángulos
for i in range(len(angles_control_free) - 1):
    angulo_actual = angles_control_free['angle (degrees)'][i]
    angulo_siguiente = angles_control_free['angle (degrees)'][i + 1]
    
    # Verificar las transiciones y contarlas
    if heading_odor[0] <= angulo_actual < heading_odor[1]:
        if opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_odor_a_opposite_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_odor_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_odor_a_heading_bottom += 1
        elif heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_odor_a_heading_odor += 1
    elif opposite_odor[0] <= angulo_actual < opposite_odor[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_opposite_odor_a_heading_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_opposite_odor_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_opposite_odor_a_heading_bottom += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_opposite_odor_a_opposite_odor += 1
    elif heading_top[0] <= angulo_actual < heading_top[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_top_a_heading_odor += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_top_a_opposite_odor += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_top_a_heading_bottom += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_top_a_heading_top += 1
    elif heading_bottom[0] <= angulo_actual <= 360 or 0 <= angulo_actual < heading_bottom[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_bottom_a_heading_odor += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_bottom_a_opposite_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_bottom_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_bottom_a_heading_bottom += 1

# Calcular las probabilidades de transición
heading_odor_transitions = (transiciones_heading_odor_a_heading_bottom, transiciones_heading_odor_a_heading_odor,
                                              transiciones_heading_odor_a_heading_top,transiciones_heading_odor_a_opposite_odor)
total_transiciones_heading_odor = sum(heading_odor_transitions)

opposite_odor_transitions = (transiciones_opposite_odor_a_heading_bottom, transiciones_opposite_odor_a_heading_odor,
                                              transiciones_opposite_odor_a_heading_top,transiciones_opposite_odor_a_opposite_odor)
total_transiciones_opposite_odor = sum(opposite_odor_transitions)

heading_top_transitions = (transiciones_heading_top_a_heading_bottom, transiciones_heading_top_a_heading_odor,
                                              transiciones_heading_top_a_heading_top,transiciones_heading_top_a_opposite_odor)
total_transiciones_heading_top = sum(heading_top_transitions)

heading_bottom_transitions = (transiciones_heading_bottom_a_heading_bottom, transiciones_heading_bottom_a_heading_odor,
                                              transiciones_heading_bottom_a_heading_top,transiciones_heading_bottom_a_opposite_odor)
total_transiciones_heading_bottom = sum(heading_bottom_transitions)

# Calcular las probabilidades de transición dividiendo el número de transiciones entre el total de transiciones

probabilidad_heading_odor_a_opposite_odor = transiciones_heading_odor_a_opposite_odor / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_top = transiciones_heading_odor_a_heading_top / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_bottom = transiciones_heading_odor_a_heading_bottom / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_odor = transiciones_heading_odor_a_heading_odor / total_transiciones_heading_odor

probabilidad_opposite_odor_a_heading_odor = transiciones_opposite_odor_a_heading_odor / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_heading_top = transiciones_opposite_odor_a_heading_top / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_heading_bottom = transiciones_opposite_odor_a_heading_bottom / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_opposite_odor = transiciones_opposite_odor_a_opposite_odor / total_transiciones_opposite_odor

probabilidad_heading_top_a_heading_odor = transiciones_heading_top_a_heading_odor / total_transiciones_heading_top
probabilidad_heading_top_a_opposite_odor = transiciones_heading_top_a_opposite_odor / total_transiciones_heading_top
probabilidad_heading_top_a_heading_bottom = transiciones_heading_top_a_heading_bottom / total_transiciones_heading_top
probabilidad_heading_top_a_heading_top = transiciones_heading_top_a_heading_top / total_transiciones_heading_top

probabilidad_heading_bottom_a_heading_odor = transiciones_heading_bottom_a_heading_odor/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_opposite_odor = transiciones_heading_bottom_a_opposite_odor/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_heading_top = transiciones_heading_bottom_a_heading_top/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_heading_bottom = transiciones_heading_bottom_a_heading_bottom/total_transiciones_heading_bottom


# Imprimir las probabilidades de transición
print(f"probabilidad_heading_odor_a_opposite_odor: {probabilidad_heading_odor_a_opposite_odor}")
print(f"probabilidad_heading_odor_a_heading_top: {probabilidad_heading_odor_a_heading_top}")
print(f"probabilidad_heading_odor_a_heading_bottom: {probabilidad_heading_odor_a_heading_bottom}")
print(f"probabilidad_heading_odor_a_heading_odor: {probabilidad_heading_odor_a_heading_odor}")

print(f"probabilidad_opposite_odor_a_heading_odor: {probabilidad_opposite_odor_a_heading_odor}")
print(f"probabilidad_opposite_odor_a_heading_top: {probabilidad_opposite_odor_a_heading_top}")
print(f"probabilidad_opposite_odor_a_heading_bottom: {probabilidad_opposite_odor_a_heading_bottom}")
print(f"probabilidad_opposite_odor_a_opposite_odor: {probabilidad_opposite_odor_a_opposite_odor}")

print(f"probabilidad_heading_top_a_heading_odor: {probabilidad_heading_top_a_heading_odor}")
print(f"probabilidad_heading_top_a_opposite_odor: {probabilidad_heading_top_a_opposite_odor}")
print(f"probabilidad_heading_top_a_heading_bottom: {probabilidad_heading_top_a_heading_bottom}")
print(f"probabilidad_heading_top_a_heading_top: {probabilidad_heading_top_a_heading_top}")

print(f"probabilidad_heading_bottom_a_heading_odor: {probabilidad_heading_bottom_a_heading_odor}")
print(f"probabilidad_heading_bottom_a_opposite_odor: {probabilidad_heading_bottom_a_opposite_odor}")
print(f"probabilidad_heading_bottom_a_heading_top: {probabilidad_heading_bottom_a_heading_top}")
print(f"probabilidad_heading_bottom_a_heading_bottom: {probabilidad_heading_bottom_a_heading_bottom}")


probabilidad_heading_odor_a_opposite_odor: 0.007658001812334536
probabilidad_heading_odor_a_heading_top: 0.05417458822693271
probabilidad_heading_odor_a_heading_bottom: 0.044366660151738596
probabilidad_heading_odor_a_heading_odor: 0.8938007498089942
probabilidad_opposite_odor_a_heading_odor: 0.0074493539639495935
probabilidad_opposite_odor_a_heading_top: 0.0530387621630244
probabilidad_opposite_odor_a_heading_bottom: 0.04169724038921678
probabilidad_opposite_odor_a_opposite_odor: 0.8978146434838092
probabilidad_heading_top_a_heading_odor: 0.04714934527804223
probabilidad_heading_top_a_opposite_odor: 0.05183150468996301
probabilidad_heading_top_a_heading_bottom: 0.008271814961060041
probabilidad_heading_top_a_heading_top: 0.8927473350709347
probabilidad_heading_bottom_a_heading_odor: 0.05196893139015325
probabilidad_heading_bottom_a_opposite_odor: 0.05541403933686892
probabilidad_heading_bottom_a_heading_top: 0.010397962166450912
probabilidad_heading_bottom_a_heading_bottom: 0.88221906

In [6]:
# Inicializa contadores para contar la población inicial en cada rango
initial_heading_odor = 0
initial_opposite_odor = 0
initial_heading_top = 0
initial_heading_bottom = 0

# Itera a través de los eventos en el tiempo 0
for angle in angles_control_free['angle (degrees)']:
    if heading_odor[0] <= angle < heading_odor[1]:
        initial_heading_odor += 1
    elif opposite_odor[0] <= angle < opposite_odor[1]:
        initial_opposite_odor += 1
    elif heading_top[0] <= angle < heading_top[1]:
        initial_heading_top += 1
    elif heading_bottom[0] <= angle <= 360 or 0 <= angle < heading_bottom[1]:
        initial_heading_bottom += 1

total_events = len(angles_control_free)

# Calcula la prob de inicio en cada rango
prob_initial_heading_odor = initial_heading_odor / total_events
prob_initial_opposite_odor = initial_opposite_odor / total_events
prob_initial_heading_top = initial_heading_top / total_events
prob_initial_heading_bottom = initial_heading_bottom / total_events

# Imprime las probes de inicio en cada rango
print("prob inicial en heading_odor:", prob_initial_heading_odor)
print("prob inicial en opposite_odor:", prob_initial_opposite_odor)
print("prob inicial en heading_top:", prob_initial_heading_top)
print("prob inicial en heading_bottom:", prob_initial_heading_bottom)

prob inicial en heading_odor: 0.2437050476532764
prob inicial en opposite_odor: 0.2714569648262095
prob inicial en heading_top: 0.27744988936472403
prob inicial en heading_bottom: 0.20738809815579007


In [11]:
# Define una función para asignar el estado según las reglas dadas
def asign_orientation(angle):
    if 225 <= angle < 315:
        return 'heading odor'
    elif 45 <= angle < 135:
        return 'opposite to odor'
    elif 135 <= angle < 225:
        return 'heading top'
    else:
        return 'heading bottom'

# Aplica la función para crear una nueva columna 'Estado'
angles_control_free['Orientation'] = angles_control_free['angle (degrees)'].apply(asign_orientation)

# Crea un diccionario para almacenar listas de ángulos por estado
angles_per_orientation = {
    'heading odor': [],
    'opposite to odor': [],
    'heading top': [],
    'heading bottom': []
}

# Llena el diccionario con los ángulos correspondientes
for orientation, sub_angles_control_free in angles_control_free.groupby('Orientation'):
    angles_per_orientation[orientation] = sub_angles_control_free['angle (degrees)'].tolist()

all_frecuencies = []

# Calcula las frecuencias absolutas y normaliza
for orientation, angles in angles_per_orientation.items():
    abs_frecuencies = pd.Series(angles).value_counts().reset_index()
    abs_frecuencies.columns = ['Angle', 'Frecuency']
    abs_frecuencies['Normalized frecuency'] = abs_frecuencies['Frecuency'] / len(angles)
    
    all_frecuencies.append({'Orientation': orientation, 'Frequencies': abs_frecuencies})
    
    print(f"Orientation: {orientation}")
    print(abs_frecuencies)
    print("\n")

Orientation: heading odor
    Angle  Frecuency  Normalized frecuency
0     247        778              0.013823
1     246        774              0.013752
2     226        768              0.013646
3     248        763              0.013557
4     249        746              0.013255
..    ...        ...                   ...
85    300        486              0.008635
86    311        481              0.008546
87    313        472              0.008386
88    310        467              0.008298
89    301        456              0.008102

[90 rows x 3 columns]


Orientation: opposite to odor
    Angle  Frecuency  Normalized frecuency
0      88        802              0.012793
1      89        796              0.012697
2      92        791              0.012618
3      90        787              0.012554
4      85        787              0.012554
..    ...        ...                   ...
85     48        621              0.009906
86     51        620              0.009890
87    133       

In [8]:
heading_odor_frec = all_frecuencies[0]
heading_odor_frec = heading_odor_frec['Frequencies']
heading_odor_frec = heading_odor_frec.sort_values(by='Angle')
heading_odor_frec = heading_odor_frec['Normalized frecuency']

heading_bottom_frec = all_frecuencies[3]
heading_bottom_frec = heading_bottom_frec['Frequencies']
heading_bottom_frec = heading_bottom_frec.sort_values(by='Angle')
heading_bottom_frec = heading_bottom_frec['Normalized frecuency']

opposite_to_odor_frec = all_frecuencies[1]
opposite_to_odor_frec = opposite_to_odor_frec['Frequencies']
opposite_to_odor_frec = opposite_to_odor_frec.sort_values(by='Angle')
opposite_to_odor_frec = opposite_to_odor_frec['Normalized frecuency']

heading_top_frec = all_frecuencies[2]
heading_top_frec = heading_top_frec['Frequencies']
heading_top_frec = heading_top_frec.sort_values(by='Angle')
heading_top_frec = heading_top_frec['Normalized frecuency']

ceros_adicionales90 = np.zeros(90)
ceros_adicionales180 = np.zeros(180)
ceros_adicionales271 = np.zeros(271)
ceros_adicionales181 = np.zeros(181)

heading_odor_frec = np.concatenate((heading_odor_frec, ceros_adicionales271))
heading_bottom_frec = np.concatenate((ceros_adicionales90,heading_bottom_frec,ceros_adicionales180))
opposite_to_odor_frec = np.concatenate((ceros_adicionales181,opposite_to_odor_frec,ceros_adicionales90))
heading_top_frec = np.concatenate((ceros_adicionales271,heading_top_frec))

# Probabilities aSyn odor

In [9]:
# Definir los rangos de ángulos
heading_odor = (225, 315)
opposite_odor = (45, 135)
heading_top = (135, 225)
heading_bottom = (315, 45)

# Inicializar contadores para contar las transiciones entre rangos y mantenerse en el mismo rango
transiciones_heading_odor_a_opposite_odor = 0
transiciones_heading_odor_a_heading_top = 0
transiciones_heading_odor_a_heading_bottom = 0
transiciones_heading_odor_a_heading_odor = 0 
transiciones_opposite_odor_a_heading_odor = 0
transiciones_opposite_odor_a_heading_top = 0
transiciones_opposite_odor_a_heading_bottom = 0
transiciones_opposite_odor_a_opposite_odor = 0 
transiciones_heading_top_a_heading_odor = 0
transiciones_heading_top_a_opposite_odor = 0
transiciones_heading_top_a_heading_bottom = 0
transiciones_heading_top_a_heading_top = 0 
transiciones_heading_bottom_a_heading_odor = 0
transiciones_heading_bottom_a_opposite_odor = 0
transiciones_heading_bottom_a_heading_top = 0
transiciones_heading_bottom_a_heading_bottom = 0  

# Iterar a través de la lista de ángulos
for i in range(len(angles_aSynA53T_odor) - 1):
    angulo_actual = angles_aSynA53T_odor['angle (degrees)'][i]
    angulo_siguiente = angles_aSynA53T_odor['angle (degrees)'][i + 1]
    
    # Verificar las transiciones y contarlas
    if heading_odor[0] <= angulo_actual < heading_odor[1]:
        if opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_odor_a_opposite_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_odor_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_odor_a_heading_bottom += 1
        elif heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_odor_a_heading_odor += 1
    elif opposite_odor[0] <= angulo_actual < opposite_odor[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_opposite_odor_a_heading_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_opposite_odor_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_opposite_odor_a_heading_bottom += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_opposite_odor_a_opposite_odor += 1
    elif heading_top[0] <= angulo_actual < heading_top[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_top_a_heading_odor += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_top_a_opposite_odor += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_top_a_heading_bottom += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_top_a_heading_top += 1
    elif heading_bottom[0] <= angulo_actual <= 360 or 0 <= angulo_actual < heading_bottom[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_bottom_a_heading_odor += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_bottom_a_opposite_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_bottom_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_bottom_a_heading_bottom += 1

# Calcular las probabilidades de transición
heading_odor_transitions = (transiciones_heading_odor_a_heading_bottom, transiciones_heading_odor_a_heading_odor,
                                              transiciones_heading_odor_a_heading_top,transiciones_heading_odor_a_opposite_odor)
total_transiciones_heading_odor = sum(heading_odor_transitions)

opposite_odor_transitions = (transiciones_opposite_odor_a_heading_bottom, transiciones_opposite_odor_a_heading_odor,
                                              transiciones_opposite_odor_a_heading_top,transiciones_opposite_odor_a_opposite_odor)
total_transiciones_opposite_odor = sum(opposite_odor_transitions)

heading_top_transitions = (transiciones_heading_top_a_heading_bottom, transiciones_heading_top_a_heading_odor,
                                              transiciones_heading_top_a_heading_top,transiciones_heading_top_a_opposite_odor)
total_transiciones_heading_top = sum(heading_top_transitions)

heading_bottom_transitions = (transiciones_heading_bottom_a_heading_bottom, transiciones_heading_bottom_a_heading_odor,
                                              transiciones_heading_bottom_a_heading_top,transiciones_heading_bottom_a_opposite_odor)
total_transiciones_heading_bottom = sum(heading_bottom_transitions)

# Calcular las probabilidades de transición dividiendo el número de transiciones entre el total de transiciones

probabilidad_heading_odor_a_opposite_odor = transiciones_heading_odor_a_opposite_odor / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_top = transiciones_heading_odor_a_heading_top / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_bottom = transiciones_heading_odor_a_heading_bottom / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_odor = transiciones_heading_odor_a_heading_odor / total_transiciones_heading_odor

probabilidad_opposite_odor_a_heading_odor = transiciones_opposite_odor_a_heading_odor / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_heading_top = transiciones_opposite_odor_a_heading_top / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_heading_bottom = transiciones_opposite_odor_a_heading_bottom / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_opposite_odor = transiciones_opposite_odor_a_opposite_odor / total_transiciones_opposite_odor

probabilidad_heading_top_a_heading_odor = transiciones_heading_top_a_heading_odor / total_transiciones_heading_top
probabilidad_heading_top_a_opposite_odor = transiciones_heading_top_a_opposite_odor / total_transiciones_heading_top
probabilidad_heading_top_a_heading_bottom = transiciones_heading_top_a_heading_bottom / total_transiciones_heading_top
probabilidad_heading_top_a_heading_top = transiciones_heading_top_a_heading_top / total_transiciones_heading_top

probabilidad_heading_bottom_a_heading_odor = transiciones_heading_bottom_a_heading_odor/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_opposite_odor = transiciones_heading_bottom_a_opposite_odor/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_heading_top = transiciones_heading_bottom_a_heading_top/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_heading_bottom = transiciones_heading_bottom_a_heading_bottom/total_transiciones_heading_bottom


# Imprimir las probabilidades de transición
print(f"probabilidad_heading_odor_a_opposite_odor: {probabilidad_heading_odor_a_opposite_odor}")
print(f"probabilidad_heading_odor_a_heading_top: {probabilidad_heading_odor_a_heading_top}")
print(f"probabilidad_heading_odor_a_heading_bottom: {probabilidad_heading_odor_a_heading_bottom}")
print(f"probabilidad_heading_odor_a_heading_odor: {probabilidad_heading_odor_a_heading_odor}")

print(f"probabilidad_opposite_odor_a_heading_odor: {probabilidad_opposite_odor_a_heading_odor}")
print(f"probabilidad_opposite_odor_a_heading_top: {probabilidad_opposite_odor_a_heading_top}")
print(f"probabilidad_opposite_odor_a_heading_bottom: {probabilidad_opposite_odor_a_heading_bottom}")
print(f"probabilidad_opposite_odor_a_opposite_odor: {probabilidad_opposite_odor_a_opposite_odor}")

print(f"probabilidad_heading_top_a_heading_odor: {probabilidad_heading_top_a_heading_odor}")
print(f"probabilidad_heading_top_a_opposite_odor: {probabilidad_heading_top_a_opposite_odor}")
print(f"probabilidad_heading_top_a_heading_bottom: {probabilidad_heading_top_a_heading_bottom}")
print(f"probabilidad_heading_top_a_heading_top: {probabilidad_heading_top_a_heading_top}")

print(f"probabilidad_heading_bottom_a_heading_odor: {probabilidad_heading_bottom_a_heading_odor}")
print(f"probabilidad_heading_bottom_a_opposite_odor: {probabilidad_heading_bottom_a_opposite_odor}")
print(f"probabilidad_heading_bottom_a_heading_top: {probabilidad_heading_bottom_a_heading_top}")
print(f"probabilidad_heading_bottom_a_heading_bottom: {probabilidad_heading_bottom_a_heading_bottom}")


probabilidad_heading_odor_a_opposite_odor: 0.00769067674100345
probabilidad_heading_odor_a_heading_top: 0.051618126096258744
probabilidad_heading_odor_a_heading_bottom: 0.051502476821957945
probabilidad_heading_odor_a_heading_odor: 0.8891887203407799
probabilidad_opposite_odor_a_heading_odor: 0.00969258398478766
probabilidad_opposite_odor_a_heading_top: 0.06071730403549546
probabilidad_opposite_odor_a_heading_bottom: 0.05427318825269385
probabilidad_opposite_odor_a_opposite_odor: 0.8753169237270231
probabilidad_heading_top_a_heading_odor: 0.06561493272692819
probabilidad_heading_top_a_opposite_odor: 0.05175762743983324
probabilidad_heading_top_a_heading_bottom: 0.009096077316657191
probabilidad_heading_top_a_heading_top: 0.8735313625165814
probabilidad_heading_bottom_a_heading_odor: 0.06617665695095679
probabilidad_heading_bottom_a_opposite_odor: 0.054162970472690405
probabilidad_heading_bottom_a_heading_top: 0.009200354834621722
probabilidad_heading_bottom_a_heading_bottom: 0.87046001

In [15]:
# Inicializa contadores para contar la población inicial en cada rango
initial_heading_odor = 0
initial_opposite_odor = 0
initial_heading_top = 0
initial_heading_bottom = 0

# Itera a través de los eventos en el tiempo 0
for angle in angles_aSynA53T_odor['angle (degrees)']:
    if heading_odor[0] <= angle < heading_odor[1]:
        initial_heading_odor += 1
    elif opposite_odor[0] <= angle < opposite_odor[1]:
        initial_opposite_odor += 1
    elif heading_top[0] <= angle < heading_top[1]:
        initial_heading_top += 1
    elif heading_bottom[0] <= angle <= 360 or 0 <= angle < heading_bottom[1]:
        initial_heading_bottom += 1

total_events = len(angles_aSynA53T_odor)

# Calcula la prob de inicio en cada rango
prob_initial_heading_odor = initial_heading_odor / total_events
prob_initial_opposite_odor = initial_opposite_odor / total_events
prob_initial_heading_top = initial_heading_top / total_events
prob_initial_heading_bottom = initial_heading_bottom / total_events

# Imprime las probes de inicio en cada rango
print("prob inicial en heading_odor:", prob_initial_heading_odor)
print("prob inicial en opposite_odor:", prob_initial_opposite_odor)
print("prob inicial en heading_top:", prob_initial_heading_top)
print("prob inicial en heading_bottom:", prob_initial_heading_bottom)

prob inicial en heading_odor: 0.30265959618940946
prob inicial en opposite_odor: 0.2208882432897554
prob inicial en heading_top: 0.2462824573992078
prob inicial en heading_bottom: 0.23016970312162738


In [16]:
# Define una función para asignar el estado según las reglas dadas
def asign_orientation(angle):
    if 225 <= angle < 315:
        return 'heading odor'
    elif 45 <= angle < 135:
        return 'opposite to odor'
    elif 135 <= angle < 225:
        return 'heading top'
    else:
        return 'heading bottom'

# Aplica la función para crear una nueva columna 'Estado'
angles_aSynA53T_odor['Orientation'] = angles_aSynA53T_odor['angle (degrees)'].apply(asign_orientation)

# Crea un diccionario para almacenar listas de ángulos por estado
angles_per_orientation = {
    'heading odor': [],
    'opposite to odor': [],
    'heading top': [],
    'heading bottom': []
}

# Llena el diccionario con los ángulos correspondientes
for orientation, sub_angles_aSynA53T_odor in angles_aSynA53T_odor.groupby('Orientation'):
    angles_per_orientation[orientation] = sub_angles_aSynA53T_odor['angle (degrees)'].tolist()

all_frecuencies = []

# Calcula las frecuencias absolutas y normaliza
for orientation, angles in angles_per_orientation.items():
    abs_frecuencies = pd.Series(angles).value_counts().reset_index()
    abs_frecuencies.columns = ['Angle', 'Frecuency']
    abs_frecuencies['Normalized frecuency'] = abs_frecuencies['Frecuency'] / len(angles)
    
    all_frecuencies.append({'Orientation': orientation, 'Frequencies': abs_frecuencies})
    
    print(f"Orientation: {orientation}")
    print(abs_frecuencies)
    print("\n")

Orientation: heading odor
    Angle  Frecuency  Normalized frecuency
0     261        681              0.013126
1     244        672              0.012953
2     262        669              0.012895
3     260        667              0.012856
4     248        663              0.012779
..    ...        ...                   ...
85    302        492              0.009483
86    304        478              0.009213
87    305        478              0.009213
88    311        475              0.009156
89    313        447              0.008616

[90 rows x 3 columns]


Orientation: opposite to odor
    Angle  Frecuency  Normalized frecuency
0      65        505              0.013337
1      62        493              0.013020
2      63        485              0.012809
3      67        477              0.012598
4      53        475              0.012545
..    ...        ...                   ...
85    110        369              0.009745
86    129        367              0.009693
87    124       

In [17]:
heading_odor_frec = all_frecuencies[0]
heading_odor_frec = heading_odor_frec['Frequencies']
heading_odor_frec = heading_odor_frec.sort_values(by='Angle')
heading_odor_frec = heading_odor_frec['Normalized frecuency']

heading_bottom_frec = all_frecuencies[3]
heading_bottom_frec = heading_bottom_frec['Frequencies']
heading_bottom_frec = heading_bottom_frec.sort_values(by='Angle')
heading_bottom_frec = heading_bottom_frec['Normalized frecuency']

opposite_to_odor_frec = all_frecuencies[1]
opposite_to_odor_frec = opposite_to_odor_frec['Frequencies']
opposite_to_odor_frec = opposite_to_odor_frec.sort_values(by='Angle')
opposite_to_odor_frec = opposite_to_odor_frec['Normalized frecuency']

heading_top_frec = all_frecuencies[2]
heading_top_frec = heading_top_frec['Frequencies']
heading_top_frec = heading_top_frec.sort_values(by='Angle')
heading_top_frec = heading_top_frec['Normalized frecuency']

ceros_adicionales90 = np.zeros(90)
ceros_adicionales180 = np.zeros(180)
ceros_adicionales271 = np.zeros(271)
ceros_adicionales181 = np.zeros(181)

heading_odor_frec = np.concatenate((heading_odor_frec, ceros_adicionales271))
heading_bottom_frec = np.concatenate((ceros_adicionales90,heading_bottom_frec,ceros_adicionales180))
opposite_to_odor_frec = np.concatenate((ceros_adicionales181,opposite_to_odor_frec,ceros_adicionales90))
heading_top_frec = np.concatenate((ceros_adicionales271,heading_top_frec))

# Probabilities asyn free

In [19]:
# Definir los rangos de ángulos
heading_odor = (225, 315)
opposite_odor = (45, 135)
heading_top = (135, 225)
heading_bottom = (315, 45)

# Inicializar contadores para contar las transiciones entre rangos y mantenerse en el mismo rango
transiciones_heading_odor_a_opposite_odor = 0
transiciones_heading_odor_a_heading_top = 0
transiciones_heading_odor_a_heading_bottom = 0
transiciones_heading_odor_a_heading_odor = 0 
transiciones_opposite_odor_a_heading_odor = 0
transiciones_opposite_odor_a_heading_top = 0
transiciones_opposite_odor_a_heading_bottom = 0
transiciones_opposite_odor_a_opposite_odor = 0 
transiciones_heading_top_a_heading_odor = 0
transiciones_heading_top_a_opposite_odor = 0
transiciones_heading_top_a_heading_bottom = 0
transiciones_heading_top_a_heading_top = 0 
transiciones_heading_bottom_a_heading_odor = 0
transiciones_heading_bottom_a_opposite_odor = 0
transiciones_heading_bottom_a_heading_top = 0
transiciones_heading_bottom_a_heading_bottom = 0  

# Iterar a través de la lista de ángulos
for i in range(len(angles_aSynA53T_free) - 1):
    angulo_actual = angles_aSynA53T_free['angle (degrees)'][i]
    angulo_siguiente = angles_aSynA53T_free['angle (degrees)'][i + 1]
    
    # Verificar las transiciones y contarlas
    if heading_odor[0] <= angulo_actual < heading_odor[1]:
        if opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_odor_a_opposite_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_odor_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_odor_a_heading_bottom += 1
        elif heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_odor_a_heading_odor += 1
    elif opposite_odor[0] <= angulo_actual < opposite_odor[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_opposite_odor_a_heading_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_opposite_odor_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_opposite_odor_a_heading_bottom += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_opposite_odor_a_opposite_odor += 1
    elif heading_top[0] <= angulo_actual < heading_top[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_top_a_heading_odor += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_top_a_opposite_odor += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_top_a_heading_bottom += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_top_a_heading_top += 1
    elif heading_bottom[0] <= angulo_actual <= 360 or 0 <= angulo_actual < heading_bottom[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_bottom_a_heading_odor += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_bottom_a_opposite_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_bottom_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_bottom_a_heading_bottom += 1

# Calcular las probabilidades de transición
heading_odor_transitions = (transiciones_heading_odor_a_heading_bottom, transiciones_heading_odor_a_heading_odor,
                                              transiciones_heading_odor_a_heading_top,transiciones_heading_odor_a_opposite_odor)
total_transiciones_heading_odor = sum(heading_odor_transitions)

opposite_odor_transitions = (transiciones_opposite_odor_a_heading_bottom, transiciones_opposite_odor_a_heading_odor,
                                              transiciones_opposite_odor_a_heading_top,transiciones_opposite_odor_a_opposite_odor)
total_transiciones_opposite_odor = sum(opposite_odor_transitions)

heading_top_transitions = (transiciones_heading_top_a_heading_bottom, transiciones_heading_top_a_heading_odor,
                                              transiciones_heading_top_a_heading_top,transiciones_heading_top_a_opposite_odor)
total_transiciones_heading_top = sum(heading_top_transitions)

heading_bottom_transitions = (transiciones_heading_bottom_a_heading_bottom, transiciones_heading_bottom_a_heading_odor,
                                              transiciones_heading_bottom_a_heading_top,transiciones_heading_bottom_a_opposite_odor)
total_transiciones_heading_bottom = sum(heading_bottom_transitions)

# Calcular las probabilidades de transición dividiendo el número de transiciones entre el total de transiciones

probabilidad_heading_odor_a_opposite_odor = transiciones_heading_odor_a_opposite_odor / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_top = transiciones_heading_odor_a_heading_top / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_bottom = transiciones_heading_odor_a_heading_bottom / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_odor = transiciones_heading_odor_a_heading_odor / total_transiciones_heading_odor

probabilidad_opposite_odor_a_heading_odor = transiciones_opposite_odor_a_heading_odor / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_heading_top = transiciones_opposite_odor_a_heading_top / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_heading_bottom = transiciones_opposite_odor_a_heading_bottom / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_opposite_odor = transiciones_opposite_odor_a_opposite_odor / total_transiciones_opposite_odor

probabilidad_heading_top_a_heading_odor = transiciones_heading_top_a_heading_odor / total_transiciones_heading_top
probabilidad_heading_top_a_opposite_odor = transiciones_heading_top_a_opposite_odor / total_transiciones_heading_top
probabilidad_heading_top_a_heading_bottom = transiciones_heading_top_a_heading_bottom / total_transiciones_heading_top
probabilidad_heading_top_a_heading_top = transiciones_heading_top_a_heading_top / total_transiciones_heading_top

probabilidad_heading_bottom_a_heading_odor = transiciones_heading_bottom_a_heading_odor/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_opposite_odor = transiciones_heading_bottom_a_opposite_odor/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_heading_top = transiciones_heading_bottom_a_heading_top/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_heading_bottom = transiciones_heading_bottom_a_heading_bottom/total_transiciones_heading_bottom


# Imprimir las probabilidades de transición
print(f"probabilidad_heading_odor_a_opposite_odor: {probabilidad_heading_odor_a_opposite_odor}")
print(f"probabilidad_heading_odor_a_heading_top: {probabilidad_heading_odor_a_heading_top}")
print(f"probabilidad_heading_odor_a_heading_bottom: {probabilidad_heading_odor_a_heading_bottom}")
print(f"probabilidad_heading_odor_a_heading_odor: {probabilidad_heading_odor_a_heading_odor}")

print(f"probabilidad_opposite_odor_a_heading_odor: {probabilidad_opposite_odor_a_heading_odor}")
print(f"probabilidad_opposite_odor_a_heading_top: {probabilidad_opposite_odor_a_heading_top}")
print(f"probabilidad_opposite_odor_a_heading_bottom: {probabilidad_opposite_odor_a_heading_bottom}")
print(f"probabilidad_opposite_odor_a_opposite_odor: {probabilidad_opposite_odor_a_opposite_odor}")

print(f"probabilidad_heading_top_a_heading_odor: {probabilidad_heading_top_a_heading_odor}")
print(f"probabilidad_heading_top_a_opposite_odor: {probabilidad_heading_top_a_opposite_odor}")
print(f"probabilidad_heading_top_a_heading_bottom: {probabilidad_heading_top_a_heading_bottom}")
print(f"probabilidad_heading_top_a_heading_top: {probabilidad_heading_top_a_heading_top}")

print(f"probabilidad_heading_bottom_a_heading_odor: {probabilidad_heading_bottom_a_heading_odor}")
print(f"probabilidad_heading_bottom_a_opposite_odor: {probabilidad_heading_bottom_a_opposite_odor}")
print(f"probabilidad_heading_bottom_a_heading_top: {probabilidad_heading_bottom_a_heading_top}")
print(f"probabilidad_heading_bottom_a_heading_bottom: {probabilidad_heading_bottom_a_heading_bottom}")


probabilidad_heading_odor_a_opposite_odor: 0.006419635671958446
probabilidad_heading_odor_a_heading_top: 0.055216182602970225
probabilidad_heading_odor_a_heading_bottom: 0.04565074255614895
probabilidad_heading_odor_a_heading_odor: 0.8927134391689224
probabilidad_opposite_odor_a_heading_odor: 0.006628620736992803
probabilidad_opposite_odor_a_heading_top: 0.05255829460100802
probabilidad_opposite_odor_a_heading_bottom: 0.04730246514090722
probabilidad_opposite_odor_a_opposite_odor: 0.893510619521092
probabilidad_heading_top_a_heading_odor: 0.05723495567758016
probabilidad_heading_top_a_opposite_odor: 0.0504882025525216
probabilidad_heading_top_a_heading_bottom: 0.004985100920182162
probabilidad_heading_top_a_heading_top: 0.8872917408497161
probabilidad_heading_bottom_a_heading_odor: 0.057024926787649595
probabilidad_heading_bottom_a_opposite_odor: 0.05499573408351973
probabilidad_heading_bottom_a_heading_top: 0.007263587520464869
probabilidad_heading_bottom_a_heading_bottom: 0.880715751

In [20]:
# Inicializa contadores para contar la población inicial en cada rango
initial_heading_odor = 0
initial_opposite_odor = 0
initial_heading_top = 0
initial_heading_bottom = 0

# Itera a través de los eventos en el tiempo 0
for angle in angles_aSynA53T_free['angle (degrees)']:
    if heading_odor[0] <= angle < heading_odor[1]:
        initial_heading_odor += 1
    elif opposite_odor[0] <= angle < opposite_odor[1]:
        initial_opposite_odor += 1
    elif heading_top[0] <= angle < heading_top[1]:
        initial_heading_top += 1
    elif heading_bottom[0] <= angle <= 360 or 0 <= angle < heading_bottom[1]:
        initial_heading_bottom += 1

total_events = len(angles_aSynA53T_free)

# Calcula la prob de inicio en cada rango
prob_initial_heading_odor = initial_heading_odor / total_events
prob_initial_opposite_odor = initial_opposite_odor / total_events
prob_initial_heading_top = initial_heading_top / total_events
prob_initial_heading_bottom = initial_heading_bottom / total_events

# Imprime las probes de inicio en cada rango
print("prob inicial en heading_odor:", prob_initial_heading_odor)
print("prob inicial en opposite_odor:", prob_initial_opposite_odor)
print("prob inicial en heading_top:", prob_initial_heading_top)
print("prob inicial en heading_bottom:", prob_initial_heading_bottom)

prob inicial en heading_odor: 0.27014634821190353
prob inicial en opposite_odor: 0.2519392867377491
prob inicial en heading_top: 0.26363923831734143
prob inicial en heading_bottom: 0.21427512673300592


In [24]:
# Define una función para asignar el estado según las reglas dadas
def asign_orientation(angle):
    if 225 <= angle < 315:
        return 'heading odor'
    elif 45 <= angle < 135:
        return 'opposite to odor'
    elif 135 <= angle < 225:
        return 'heading top'
    else:
        return 'heading bottom'

# Aplica la función para crear una nueva columna 'Estado'
angles_aSynA53T_free['Orientation'] = angles_aSynA53T_free['angle (degrees)'].apply(asign_orientation)

# Crea un diccionario para almacenar listas de ángulos por estado
angles_per_orientation = {
    'heading odor': [],
    'opposite to odor': [],
    'heading top': [],
    'heading bottom': []
}

# Llena el diccionario con los ángulos correspondientes
for orientation, sub_angles_aSynA53T_free in angles_aSynA53T_free.groupby('Orientation'):
    angles_per_orientation[orientation] = sub_angles_aSynA53T_free['angle (degrees)'].tolist()

all_frecuencies = []

# Calcula las frecuencias absolutas y normaliza
for orientation, angles in angles_per_orientation.items():
    abs_frecuencies = pd.Series(angles).value_counts().reset_index()
    abs_frecuencies.columns = ['Angle', 'Frecuency']
    abs_frecuencies['Normalized frecuency'] = abs_frecuencies['Frecuency'] / len(angles)
    
    all_frecuencies.append({'Orientation': orientation, 'Frequencies': abs_frecuencies})
    
    print(f"Orientation: {orientation}")
    print(abs_frecuencies)
    print("\n")

Orientation: heading odor
    Angle  Frecuency  Normalized frecuency
0     249        751              0.013735
1     238        736              0.013461
2     225        733              0.013406
3     228        731              0.013370
4     241        722              0.013205
..    ...        ...                   ...
85    310        462              0.008450
86    301        462              0.008450
87    309        461              0.008431
88    307        460              0.008413
89    308        437              0.007993

[90 rows x 3 columns]


Orientation: opposite to odor
    Angle  Frecuency  Normalized frecuency
0      76        665              0.013042
1      74        658              0.012904
2      71        653              0.012806
3      70        649              0.012728
4      69        635              0.012453
..    ...        ...                   ...
85    125        510              0.010002
86    133        501              0.009825
87    126       

In [25]:
heading_odor_frec = all_frecuencies[0]
heading_odor_frec = heading_odor_frec['Frequencies']
heading_odor_frec = heading_odor_frec.sort_values(by='Angle')
heading_odor_frec = heading_odor_frec['Normalized frecuency']

heading_bottom_frec = all_frecuencies[3]
heading_bottom_frec = heading_bottom_frec['Frequencies']
heading_bottom_frec = heading_bottom_frec.sort_values(by='Angle')
heading_bottom_frec = heading_bottom_frec['Normalized frecuency']

opposite_to_odor_frec = all_frecuencies[1]
opposite_to_odor_frec = opposite_to_odor_frec['Frequencies']
opposite_to_odor_frec = opposite_to_odor_frec.sort_values(by='Angle')
opposite_to_odor_frec = opposite_to_odor_frec['Normalized frecuency']

heading_top_frec = all_frecuencies[2]
heading_top_frec = heading_top_frec['Frequencies']
heading_top_frec = heading_top_frec.sort_values(by='Angle')
heading_top_frec = heading_top_frec['Normalized frecuency']

ceros_adicionales90 = np.zeros(90)
ceros_adicionales180 = np.zeros(180)
ceros_adicionales271 = np.zeros(271)
ceros_adicionales181 = np.zeros(181)

heading_odor_frec = np.concatenate((heading_odor_frec, ceros_adicionales271))
heading_bottom_frec = np.concatenate((ceros_adicionales90,heading_bottom_frec,ceros_adicionales180))
opposite_to_odor_frec = np.concatenate((ceros_adicionales181,opposite_to_odor_frec,ceros_adicionales90))
heading_top_frec = np.concatenate((ceros_adicionales271,heading_top_frec))

# Probabilities LRRK2 odor

In [13]:
# Definir los rangos de ángulos
heading_odor = (225, 315)
opposite_odor = (45, 135)
heading_top = (135, 225)
heading_bottom = (315, 45)

# Inicializar contadores para contar las transiciones entre rangos y mantenerse en el mismo rango
transiciones_heading_odor_a_opposite_odor = 0
transiciones_heading_odor_a_heading_top = 0
transiciones_heading_odor_a_heading_bottom = 0
transiciones_heading_odor_a_heading_odor = 0 
transiciones_opposite_odor_a_heading_odor = 0
transiciones_opposite_odor_a_heading_top = 0
transiciones_opposite_odor_a_heading_bottom = 0
transiciones_opposite_odor_a_opposite_odor = 0 
transiciones_heading_top_a_heading_odor = 0
transiciones_heading_top_a_opposite_odor = 0
transiciones_heading_top_a_heading_bottom = 0
transiciones_heading_top_a_heading_top = 0 
transiciones_heading_bottom_a_heading_odor = 0
transiciones_heading_bottom_a_opposite_odor = 0
transiciones_heading_bottom_a_heading_top = 0
transiciones_heading_bottom_a_heading_bottom = 0  

# Iterar a través de la lista de ángulos
for i in range(len(angles_G2019S_odor) - 1):
    angulo_actual = angles_G2019S_odor['angle (degrees)'][i]
    angulo_siguiente = angles_G2019S_odor['angle (degrees)'][i + 1]
    
    # Verificar las transiciones y contarlas
    if heading_odor[0] <= angulo_actual < heading_odor[1]:
        if opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_odor_a_opposite_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_odor_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_odor_a_heading_bottom += 1
        elif heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_odor_a_heading_odor += 1
    elif opposite_odor[0] <= angulo_actual < opposite_odor[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_opposite_odor_a_heading_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_opposite_odor_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_opposite_odor_a_heading_bottom += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_opposite_odor_a_opposite_odor += 1
    elif heading_top[0] <= angulo_actual < heading_top[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_top_a_heading_odor += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_top_a_opposite_odor += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_top_a_heading_bottom += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_top_a_heading_top += 1
    elif heading_bottom[0] <= angulo_actual <= 360 or 0 <= angulo_actual < heading_bottom[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_bottom_a_heading_odor += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_bottom_a_opposite_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_bottom_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_bottom_a_heading_bottom += 1

# Calcular las probabilidades de transición
heading_odor_transitions = (transiciones_heading_odor_a_heading_bottom, transiciones_heading_odor_a_heading_odor,
                                              transiciones_heading_odor_a_heading_top,transiciones_heading_odor_a_opposite_odor)
total_transiciones_heading_odor = sum(heading_odor_transitions)

opposite_odor_transitions = (transiciones_opposite_odor_a_heading_bottom, transiciones_opposite_odor_a_heading_odor,
                                              transiciones_opposite_odor_a_heading_top,transiciones_opposite_odor_a_opposite_odor)
total_transiciones_opposite_odor = sum(opposite_odor_transitions)

heading_top_transitions = (transiciones_heading_top_a_heading_bottom, transiciones_heading_top_a_heading_odor,
                                              transiciones_heading_top_a_heading_top,transiciones_heading_top_a_opposite_odor)
total_transiciones_heading_top = sum(heading_top_transitions)

heading_bottom_transitions = (transiciones_heading_bottom_a_heading_bottom, transiciones_heading_bottom_a_heading_odor,
                                              transiciones_heading_bottom_a_heading_top,transiciones_heading_bottom_a_opposite_odor)
total_transiciones_heading_bottom = sum(heading_bottom_transitions)

# Calcular las probabilidades de transición dividiendo el número de transiciones entre el total de transiciones

probabilidad_heading_odor_a_opposite_odor = transiciones_heading_odor_a_opposite_odor / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_top = transiciones_heading_odor_a_heading_top / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_bottom = transiciones_heading_odor_a_heading_bottom / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_odor = transiciones_heading_odor_a_heading_odor / total_transiciones_heading_odor

probabilidad_opposite_odor_a_heading_odor = transiciones_opposite_odor_a_heading_odor / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_heading_top = transiciones_opposite_odor_a_heading_top / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_heading_bottom = transiciones_opposite_odor_a_heading_bottom / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_opposite_odor = transiciones_opposite_odor_a_opposite_odor / total_transiciones_opposite_odor

probabilidad_heading_top_a_heading_odor = transiciones_heading_top_a_heading_odor / total_transiciones_heading_top
probabilidad_heading_top_a_opposite_odor = transiciones_heading_top_a_opposite_odor / total_transiciones_heading_top
probabilidad_heading_top_a_heading_bottom = transiciones_heading_top_a_heading_bottom / total_transiciones_heading_top
probabilidad_heading_top_a_heading_top = transiciones_heading_top_a_heading_top / total_transiciones_heading_top

probabilidad_heading_bottom_a_heading_odor = transiciones_heading_bottom_a_heading_odor/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_opposite_odor = transiciones_heading_bottom_a_opposite_odor/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_heading_top = transiciones_heading_bottom_a_heading_top/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_heading_bottom = transiciones_heading_bottom_a_heading_bottom/total_transiciones_heading_bottom


# Imprimir las probabilidades de transición
print(f"probabilidad_heading_odor_a_opposite_odor: {probabilidad_heading_odor_a_opposite_odor}")
print(f"probabilidad_heading_odor_a_heading_top: {probabilidad_heading_odor_a_heading_top}")
print(f"probabilidad_heading_odor_a_heading_bottom: {probabilidad_heading_odor_a_heading_bottom}")
print(f"probabilidad_heading_odor_a_heading_odor: {probabilidad_heading_odor_a_heading_odor}")

print(f"probabilidad_opposite_odor_a_heading_odor: {probabilidad_opposite_odor_a_heading_odor}")
print(f"probabilidad_opposite_odor_a_heading_top: {probabilidad_opposite_odor_a_heading_top}")
print(f"probabilidad_opposite_odor_a_heading_bottom: {probabilidad_opposite_odor_a_heading_bottom}")
print(f"probabilidad_opposite_odor_a_opposite_odor: {probabilidad_opposite_odor_a_opposite_odor}")

print(f"probabilidad_heading_top_a_heading_odor: {probabilidad_heading_top_a_heading_odor}")
print(f"probabilidad_heading_top_a_opposite_odor: {probabilidad_heading_top_a_opposite_odor}")
print(f"probabilidad_heading_top_a_heading_bottom: {probabilidad_heading_top_a_heading_bottom}")
print(f"probabilidad_heading_top_a_heading_top: {probabilidad_heading_top_a_heading_top}")

print(f"probabilidad_heading_bottom_a_heading_odor: {probabilidad_heading_bottom_a_heading_odor}")
print(f"probabilidad_heading_bottom_a_opposite_odor: {probabilidad_heading_bottom_a_opposite_odor}")
print(f"probabilidad_heading_bottom_a_heading_top: {probabilidad_heading_bottom_a_heading_top}")
print(f"probabilidad_heading_bottom_a_heading_bottom: {probabilidad_heading_bottom_a_heading_bottom}")


probabilidad_heading_odor_a_opposite_odor: 0.0056081573197378
probabilidad_heading_odor_a_heading_top: 0.05416970138383103
probabilidad_heading_odor_a_heading_bottom: 0.04515659140568099
probabilidad_heading_odor_a_heading_odor: 0.8950655498907502
probabilidad_opposite_odor_a_heading_odor: 0.007275390625
probabilidad_opposite_odor_a_heading_top: 0.0573486328125
probabilidad_opposite_odor_a_heading_bottom: 0.05283203125
probabilidad_opposite_odor_a_opposite_odor: 0.8825439453125
probabilidad_heading_top_a_heading_odor: 0.06435420554733252
probabilidad_heading_top_a_opposite_odor: 0.04978870369178626
probabilidad_heading_top_a_heading_bottom: 0.006328163545487698
probabilidad_heading_top_a_heading_top: 0.8795289272153936
probabilidad_heading_bottom_a_heading_odor: 0.06254598969830757
probabilidad_heading_bottom_a_opposite_odor: 0.05539062698231458
probabilidad_heading_bottom_a_heading_top: 0.007383725355865114
probabilidad_heading_bottom_a_heading_bottom: 0.8746796579635128


In [31]:
# Inicializa contadores para contar la población inicial en cada rango
initial_heading_odor = 0
initial_opposite_odor = 0
initial_heading_top = 0
initial_heading_bottom = 0

# Itera a través de los eventos en el tiempo 0
for angle in angles_G2019S_odor['angle (degrees)']:
    if heading_odor[0] <= angle < heading_odor[1]:
        initial_heading_odor += 1
    elif opposite_odor[0] <= angle < opposite_odor[1]:
        initial_opposite_odor += 1
    elif heading_top[0] <= angle < heading_top[1]:
        initial_heading_top += 1
    elif heading_bottom[0] <= angle <= 360 or 0 <= angle < heading_bottom[1]:
        initial_heading_bottom += 1

total_events = len(angles_G2019S_odor)

# Calcula la prob de inicio en cada rango
prob_initial_heading_odor = initial_heading_odor / total_events
prob_initial_opposite_odor = initial_opposite_odor / total_events
prob_initial_heading_top = initial_heading_top / total_events
prob_initial_heading_bottom = initial_heading_bottom / total_events

# Imprime las probes de inicio en cada rango
print("prob inicial en heading_odor:", prob_initial_heading_odor)
print("prob inicial en opposite_odor:", prob_initial_opposite_odor)
print("prob inicial en heading_top:", prob_initial_heading_top)
print("prob inicial en heading_bottom:", prob_initial_heading_bottom)

prob inicial en heading_odor: 0.30190919635642
prob inicial en opposite_odor: 0.2251730260734763
prob inicial en heading_top: 0.25626549538505516
prob inicial en heading_bottom: 0.21665228218504856


In [32]:
# Define una función para asignar el estado según las reglas dadas
def asign_orientation(angle):
    if 225 <= angle < 315:
        return 'heading odor'
    elif 45 <= angle < 135:
        return 'opposite to odor'
    elif 135 <= angle < 225:
        return 'heading top'
    else:
        return 'heading bottom'

# Aplica la función para crear una nueva columna 'Estado'
angles_G2019S_odor['Orientation'] = angles_G2019S_odor['angle (degrees)'].apply(asign_orientation)

# Crea un diccionario para almacenar listas de ángulos por estado
angles_per_orientation = {
    'heading odor': [],
    'opposite to odor': [],
    'heading top': [],
    'heading bottom': []
}

# Llena el diccionario con los ángulos correspondientes
for orientation, sub_angles_G2019S_odor in angles_G2019S_odor.groupby('Orientation'):
    angles_per_orientation[orientation] = sub_angles_G2019S_odor['angle (degrees)'].tolist()

all_frecuencies = []

# Calcula las frecuencias absolutas y normaliza
for orientation, angles in angles_per_orientation.items():
    abs_frecuencies = pd.Series(angles).value_counts().reset_index()
    abs_frecuencies.columns = ['Angle', 'Frecuency']
    abs_frecuencies['Normalized frecuency'] = abs_frecuencies['Frecuency'] / len(angles)
    
    all_frecuencies.append({'Orientation': orientation, 'Frequencies': abs_frecuencies})
    
    print(f"Orientation: {orientation}")
    print(abs_frecuencies)
    print("\n")

Orientation: heading odor
    Angle  Frecuency  Normalized frecuency
0     248        759              0.013820
1     249        754              0.013729
2     250        733              0.013347
3     247        731              0.013310
4     257        729              0.013274
..    ...        ...                   ...
85    301        481              0.008758
86    304        476              0.008667
87    311        473              0.008613
88    309        470              0.008558
89    313        429              0.007811

[90 rows x 3 columns]


Orientation: opposite to odor
    Angle  Frecuency  Normalized frecuency
0      82        525              0.012817
1      70        521              0.012719
2      60        520              0.012695
3      86        515              0.012573
4      46        513              0.012524
..    ...        ...                   ...
85    100        390              0.009521
86    129        384              0.009375
87    131       

In [33]:
heading_odor_frec = all_frecuencies[0]
heading_odor_frec = heading_odor_frec['Frequencies']
heading_odor_frec = heading_odor_frec.sort_values(by='Angle')
heading_odor_frec = heading_odor_frec['Normalized frecuency']

heading_bottom_frec = all_frecuencies[3]
heading_bottom_frec = heading_bottom_frec['Frequencies']
heading_bottom_frec = heading_bottom_frec.sort_values(by='Angle')
heading_bottom_frec = heading_bottom_frec['Normalized frecuency']

opposite_to_odor_frec = all_frecuencies[1]
opposite_to_odor_frec = opposite_to_odor_frec['Frequencies']
opposite_to_odor_frec = opposite_to_odor_frec.sort_values(by='Angle')
opposite_to_odor_frec = opposite_to_odor_frec['Normalized frecuency']

heading_top_frec = all_frecuencies[2]
heading_top_frec = heading_top_frec['Frequencies']
heading_top_frec = heading_top_frec.sort_values(by='Angle')
heading_top_frec = heading_top_frec['Normalized frecuency']

ceros_adicionales90 = np.zeros(90)
ceros_adicionales180 = np.zeros(180)
ceros_adicionales271 = np.zeros(271)
ceros_adicionales181 = np.zeros(181)

heading_odor_frec = np.concatenate((heading_odor_frec, ceros_adicionales271))
heading_bottom_frec = np.concatenate((ceros_adicionales90,heading_bottom_frec,ceros_adicionales180))
opposite_to_odor_frec = np.concatenate((ceros_adicionales181,opposite_to_odor_frec,ceros_adicionales90))
heading_top_frec = np.concatenate((ceros_adicionales271,heading_top_frec))

# Probabilities G2019S free

In [29]:
# Definir los rangos de ángulos
heading_odor = (225, 315)
opposite_odor = (45, 135)
heading_top = (135, 225)
heading_bottom = (315, 45)

# Inicializar contadores para contar las transiciones entre rangos y mantenerse en el mismo rango
transiciones_heading_odor_a_opposite_odor = 0
transiciones_heading_odor_a_heading_top = 0
transiciones_heading_odor_a_heading_bottom = 0
transiciones_heading_odor_a_heading_odor = 0 
transiciones_opposite_odor_a_heading_odor = 0
transiciones_opposite_odor_a_heading_top = 0
transiciones_opposite_odor_a_heading_bottom = 0
transiciones_opposite_odor_a_opposite_odor = 0 
transiciones_heading_top_a_heading_odor = 0
transiciones_heading_top_a_opposite_odor = 0
transiciones_heading_top_a_heading_bottom = 0
transiciones_heading_top_a_heading_top = 0 
transiciones_heading_bottom_a_heading_odor = 0
transiciones_heading_bottom_a_opposite_odor = 0
transiciones_heading_bottom_a_heading_top = 0
transiciones_heading_bottom_a_heading_bottom = 0  

# Iterar a través de la lista de ángulos
for i in range(len(angles_G2019S_free) - 1):
    angulo_actual = angles_G2019S_free['angle (degrees)'][i]
    angulo_siguiente = angles_G2019S_free['angle (degrees)'][i + 1]
    
    # Verificar las transiciones y contarlas
    if heading_odor[0] <= angulo_actual < heading_odor[1]:
        if opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_odor_a_opposite_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_odor_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_odor_a_heading_bottom += 1
        elif heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_odor_a_heading_odor += 1
    elif opposite_odor[0] <= angulo_actual < opposite_odor[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_opposite_odor_a_heading_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_opposite_odor_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_opposite_odor_a_heading_bottom += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_opposite_odor_a_opposite_odor += 1
    elif heading_top[0] <= angulo_actual < heading_top[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_top_a_heading_odor += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_top_a_opposite_odor += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_top_a_heading_bottom += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_top_a_heading_top += 1
    elif heading_bottom[0] <= angulo_actual <= 360 or 0 <= angulo_actual < heading_bottom[1]:
        if heading_odor[0] <= angulo_siguiente < heading_odor[1]:
            transiciones_heading_bottom_a_heading_odor += 1
        elif opposite_odor[0] <= angulo_siguiente < opposite_odor[1]:
            transiciones_heading_bottom_a_opposite_odor += 1
        elif heading_top[0] <= angulo_siguiente < heading_top[1]:
            transiciones_heading_bottom_a_heading_top += 1
        elif heading_bottom[0] <= angulo_siguiente <= 360 or 0 <= angulo_siguiente < heading_bottom[1]:
            transiciones_heading_bottom_a_heading_bottom += 1

# Calcular las probabilidades de transición
heading_odor_transitions = (transiciones_heading_odor_a_heading_bottom, transiciones_heading_odor_a_heading_odor,
                                              transiciones_heading_odor_a_heading_top,transiciones_heading_odor_a_opposite_odor)
total_transiciones_heading_odor = sum(heading_odor_transitions)

opposite_odor_transitions = (transiciones_opposite_odor_a_heading_bottom, transiciones_opposite_odor_a_heading_odor,
                                              transiciones_opposite_odor_a_heading_top,transiciones_opposite_odor_a_opposite_odor)
total_transiciones_opposite_odor = sum(opposite_odor_transitions)

heading_top_transitions = (transiciones_heading_top_a_heading_bottom, transiciones_heading_top_a_heading_odor,
                                              transiciones_heading_top_a_heading_top,transiciones_heading_top_a_opposite_odor)
total_transiciones_heading_top = sum(heading_top_transitions)

heading_bottom_transitions = (transiciones_heading_bottom_a_heading_bottom, transiciones_heading_bottom_a_heading_odor,
                                              transiciones_heading_bottom_a_heading_top,transiciones_heading_bottom_a_opposite_odor)
total_transiciones_heading_bottom = sum(heading_bottom_transitions)

# Calcular las probabilidades de transición dividiendo el número de transiciones entre el total de transiciones

probabilidad_heading_odor_a_opposite_odor = transiciones_heading_odor_a_opposite_odor / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_top = transiciones_heading_odor_a_heading_top / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_bottom = transiciones_heading_odor_a_heading_bottom / total_transiciones_heading_odor
probabilidad_heading_odor_a_heading_odor = transiciones_heading_odor_a_heading_odor / total_transiciones_heading_odor

probabilidad_opposite_odor_a_heading_odor = transiciones_opposite_odor_a_heading_odor / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_heading_top = transiciones_opposite_odor_a_heading_top / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_heading_bottom = transiciones_opposite_odor_a_heading_bottom / total_transiciones_opposite_odor
probabilidad_opposite_odor_a_opposite_odor = transiciones_opposite_odor_a_opposite_odor / total_transiciones_opposite_odor

probabilidad_heading_top_a_heading_odor = transiciones_heading_top_a_heading_odor / total_transiciones_heading_top
probabilidad_heading_top_a_opposite_odor = transiciones_heading_top_a_opposite_odor / total_transiciones_heading_top
probabilidad_heading_top_a_heading_bottom = transiciones_heading_top_a_heading_bottom / total_transiciones_heading_top
probabilidad_heading_top_a_heading_top = transiciones_heading_top_a_heading_top / total_transiciones_heading_top

probabilidad_heading_bottom_a_heading_odor = transiciones_heading_bottom_a_heading_odor/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_opposite_odor = transiciones_heading_bottom_a_opposite_odor/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_heading_top = transiciones_heading_bottom_a_heading_top/total_transiciones_heading_bottom
probabilidad_heading_bottom_a_heading_bottom = transiciones_heading_bottom_a_heading_bottom/total_transiciones_heading_bottom


# Imprimir las probabilidades de transición
print(f"probabilidad_heading_odor_a_opposite_odor: {probabilidad_heading_odor_a_opposite_odor}")
print(f"probabilidad_heading_odor_a_heading_top: {probabilidad_heading_odor_a_heading_top}")
print(f"probabilidad_heading_odor_a_heading_bottom: {probabilidad_heading_odor_a_heading_bottom}")
print(f"probabilidad_heading_odor_a_heading_odor: {probabilidad_heading_odor_a_heading_odor}")

print(f"probabilidad_opposite_odor_a_heading_odor: {probabilidad_opposite_odor_a_heading_odor}")
print(f"probabilidad_opposite_odor_a_heading_top: {probabilidad_opposite_odor_a_heading_top}")
print(f"probabilidad_opposite_odor_a_heading_bottom: {probabilidad_opposite_odor_a_heading_bottom}")
print(f"probabilidad_opposite_odor_a_opposite_odor: {probabilidad_opposite_odor_a_opposite_odor}")

print(f"probabilidad_heading_top_a_heading_odor: {probabilidad_heading_top_a_heading_odor}")
print(f"probabilidad_heading_top_a_opposite_odor: {probabilidad_heading_top_a_opposite_odor}")
print(f"probabilidad_heading_top_a_heading_bottom: {probabilidad_heading_top_a_heading_bottom}")
print(f"probabilidad_heading_top_a_heading_top: {probabilidad_heading_top_a_heading_top}")

print(f"probabilidad_heading_bottom_a_heading_odor: {probabilidad_heading_bottom_a_heading_odor}")
print(f"probabilidad_heading_bottom_a_opposite_odor: {probabilidad_heading_bottom_a_opposite_odor}")
print(f"probabilidad_heading_bottom_a_heading_top: {probabilidad_heading_bottom_a_heading_top}")
print(f"probabilidad_heading_bottom_a_heading_bottom: {probabilidad_heading_bottom_a_heading_bottom}")


probabilidad_heading_odor_a_opposite_odor: 0.012457555637260203
probabilidad_heading_odor_a_heading_top: 0.07163094491424617
probabilidad_heading_odor_a_heading_bottom: 0.052836472954560204
probabilidad_heading_odor_a_heading_odor: 0.8630750264939334
probabilidad_opposite_odor_a_heading_odor: 0.011807447774750226
probabilidad_opposite_odor_a_heading_top: 0.06284059945504088
probabilidad_opposite_odor_a_heading_bottom: 0.05249016046018771
probabilidad_opposite_odor_a_opposite_odor: 0.8728617923100211
probabilidad_heading_top_a_heading_odor: 0.05705113666102552
probabilidad_heading_top_a_opposite_odor: 0.05815028700036638
probabilidad_heading_top_a_heading_bottom: 0.009456182284488022
probabilidad_heading_top_a_heading_top: 0.8753423940541201
probabilidad_heading_bottom_a_heading_odor: 0.05925979963038615
probabilidad_heading_bottom_a_opposite_odor: 0.06835424569594398
probabilidad_heading_bottom_a_heading_top: 0.012450150763544403
probabilidad_heading_bottom_a_heading_bottom: 0.85993580

In [39]:
# Inicializa contadores para contar la población inicial en cada rango
initial_heading_odor = 0
initial_opposite_odor = 0
initial_heading_top = 0
initial_heading_bottom = 0

# Itera a través de los eventos en el tiempo 0
for angle in angles_G2019S_free['angle (degrees)']:
    if heading_odor[0] <= angle < heading_odor[1]:
        initial_heading_odor += 1
    elif opposite_odor[0] <= angle < opposite_odor[1]:
        initial_opposite_odor += 1
    elif heading_top[0] <= angle < heading_top[1]:
        initial_heading_top += 1
    elif heading_bottom[0] <= angle <= 360 or 0 <= angle < heading_bottom[1]:
        initial_heading_bottom += 1

total_events = len(angles_G2019S_free)

# Calcula la prob de inicio en cada rango
prob_initial_heading_odor = initial_heading_odor / total_events
prob_initial_opposite_odor = initial_opposite_odor / total_events
prob_initial_heading_top = initial_heading_top / total_events
prob_initial_heading_bottom = initial_heading_bottom / total_events

# Imprime las probes de inicio en cada rango
print("prob inicial en heading_odor:", prob_initial_heading_odor)
print("prob inicial en opposite_odor:", prob_initial_opposite_odor)
print("prob inicial en heading_top:", prob_initial_heading_top)
print("prob inicial en heading_bottom:", prob_initial_heading_bottom)

prob inicial en heading_odor: 0.2340793916780997
prob inicial en opposite_odor: 0.267553296511363
prob inicial en heading_top: 0.2901729890090975
prob inicial en heading_bottom: 0.2081943228014398


In [40]:
# Define una función para asignar el estado según las reglas dadas
def asign_orientation(angle):
    if 225 <= angle < 315:
        return 'heading odor'
    elif 45 <= angle < 135:
        return 'opposite to odor'
    elif 135 <= angle < 225:
        return 'heading top'
    else:
        return 'heading bottom'

# Aplica la función para crear una nueva columna 'Estado'
angles_G2019S_free['Orientation'] = angles_G2019S_free['angle (degrees)'].apply(asign_orientation)

# Crea un diccionario para almacenar listas de ángulos por estado
angles_per_orientation = {
    'heading odor': [],
    'opposite to odor': [],
    'heading top': [],
    'heading bottom': []
}

# Llena el diccionario con los ángulos correspondientes
for orientation, sub_angles_G2019S_free in angles_G2019S_free.groupby('Orientation'):
    angles_per_orientation[orientation] = sub_angles_G2019S_free['angle (degrees)'].tolist()

all_frecuencies = []

# Calcula las frecuencias absolutas y normaliza
for orientation, angles in angles_per_orientation.items():
    abs_frecuencies = pd.Series(angles).value_counts().reset_index()
    abs_frecuencies.columns = ['Angle', 'Frecuency']
    abs_frecuencies['Normalized frecuency'] = abs_frecuencies['Frecuency'] / len(angles)
    
    all_frecuencies.append({'Orientation': orientation, 'Frequencies': abs_frecuencies})
    
    print(f"Orientation: {orientation}")
    print(abs_frecuencies)
    print("\n")

Orientation: heading odor
    Angle  Frecuency  Normalized frecuency
0     226        674              0.014577
1     228        648              0.014015
2     225        631              0.013647
3     234        621              0.013431
4     243        614              0.013279
..    ...        ...                   ...
85    292        400              0.008651
86    300        397              0.008586
87    314        394              0.008521
88    305        394              0.008521
89    313        367              0.007937

[90 rows x 3 columns]


Orientation: opposite to odor
    Angle  Frecuency  Normalized frecuency
0      75        683              0.012924
1      99        657              0.012432
2      86        656              0.012413
3      77        649              0.012280
4      78        645              0.012205
..    ...        ...                   ...
85     48        518              0.009802
86     49        518              0.009802
87    124       

In [41]:
heading_odor_frec = all_frecuencies[0]
heading_odor_frec = heading_odor_frec['Frequencies']
heading_odor_frec = heading_odor_frec.sort_values(by='Angle')
heading_odor_frec = heading_odor_frec['Normalized frecuency']

heading_bottom_frec = all_frecuencies[3]
heading_bottom_frec = heading_bottom_frec['Frequencies']
heading_bottom_frec = heading_bottom_frec.sort_values(by='Angle')
heading_bottom_frec = heading_bottom_frec['Normalized frecuency']

opposite_to_odor_frec = all_frecuencies[1]
opposite_to_odor_frec = opposite_to_odor_frec['Frequencies']
opposite_to_odor_frec = opposite_to_odor_frec.sort_values(by='Angle')
opposite_to_odor_frec = opposite_to_odor_frec['Normalized frecuency']

heading_top_frec = all_frecuencies[2]
heading_top_frec = heading_top_frec['Frequencies']
heading_top_frec = heading_top_frec.sort_values(by='Angle')
heading_top_frec = heading_top_frec['Normalized frecuency']

ceros_adicionales90 = np.zeros(90)
ceros_adicionales180 = np.zeros(180)
ceros_adicionales271 = np.zeros(271)
ceros_adicionales181 = np.zeros(181)

heading_odor_frec = np.concatenate((heading_odor_frec, ceros_adicionales271))
heading_bottom_frec = np.concatenate((ceros_adicionales90,heading_bottom_frec,ceros_adicionales180))
opposite_to_odor_frec = np.concatenate((ceros_adicionales181,opposite_to_odor_frec,ceros_adicionales90))
heading_top_frec = np.concatenate((ceros_adicionales271,heading_top_frec))